# 🚀 Self-Correction Human Parsing - Colab 演示

**人体解析（Human Parsing）** - 识别和分割人体的不同部位

⚠️ **重要提示**:
1. 点击 **Runtime → Change runtime type**
2. 选择 **GPU** 作为硬件加速器
3. 保存设置后开始运行

---

## 支持的数据集

| 数据集 | 标签数 | mIoU | 特点 |
|--------|--------|------|------|
| **LIP** | 20个 | 59.36% | 最全面，适合复杂场景 |
| **ATR** | 18个 | 82.29% | 适合时尚AI |
| **Pascal** | 7个 | 71.46% | 简化版，身体部位 |


In [1]:
# ========== 步骤 1: 安装兼容版本的 PyTorch ==========
print('📦 安装 PyTorch 1.7.1 (兼容版本)...')
print('⏳ 预计需要 2-3 分钟...\n')

!pip install -q torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q ninja

print('✅ PyTorch 安装完成！')

# 验证安装
import torch
print(f'🔥 PyTorch 版本: {torch.__version__}')
print(f'🎮 CUDA 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'🚀 GPU: {torch.cuda.get_device_name(0)}')


📦 安装 PyTorch 1.7.1 (兼容版本)...
⏳ 预计需要 2-3 分钟...

ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.0+rocm5.7, 2.2.1, 2.2.1+cpu, 2.2.1+cpu.cxx11.abi, 2.2.1+cu118, 2.2.1+cu121, 2.2.1+rocm5.6, 2.2.1+rocm5.7, 2.2.2, 2.2.2+cpu, 2.2.2+cpu.cxx11.abi, 2.2.2+cu118, 2.2.2+cu121, 2.2.2+rocm5.6, 2.2.2+rocm5.7, 2.3.0, 2.3.0+cpu, 2.3.0+cpu.cxx11.abi, 2.3.0+cu118, 2.3.0+cu121, 2.3.0+rocm5.7, 2.3.0+rocm6.0, 2.3.1, 2.3.1+cpu, 2.3.1+cpu.cxx11.abi, 2.3.1+cu118, 2.3.1+cu121, 2.3.1+rocm5.7, 2.3.1+rocm6.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0)
ERROR: No matching distribution found for torch==1.7.1+cu101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 8.8 MB/s eta 0:00:00
✅ PyTorch 安装完成！
🔥 PyTorch 版本: 2.8.0+cu126
🎮 CUDA 可用: True
🚀 GPU: Tesla T4


In [2]:
# ========== 步骤 2: 克隆项目 ==========
print('📥 克隆项目代码...')

!git clone https://github.com/likunpeng0127/salf.git
%cd salf

# 创建必要的文件夹
!mkdir -p checkpoints inputs outputs

print('✅ 项目准备完成！')


📥 克隆项目代码...
Cloning into 'salf'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'salf'
/content
✅ 项目准备完成！


In [3]:
# ========== 步骤 3: 选择数据集 ==========
# 请修改下面的 dataset 变量选择数据集
# 可选值: 'lip', 'atr', 'pascal'

dataset = 'lip'  # 👈 修改这里选择数据集

# 数据集信息
dataset_info = {
    'lip': {
        'name': 'LIP (Look Into Person)',
        'labels': 20,
        'miou': '59.36%',
        'desc': '最大数据集，包含帽子、头发、手套、上衣、裤子等20个部位'
    },
    'atr': {
        'name': 'ATR (A person re-identification dataset)',
        'labels': 18,
        'miou': '82.29%',
        'desc': '时尚AI专用，包含包包、腰带等18个部位'
    },
    'pascal': {
        'name': 'Pascal-Person-Part',
        'labels': 7,
        'miou': '71.46%',
        'desc': '简化版，只包含头、躯干、手臂、腿等7个部位'
    }
}

info = dataset_info[dataset]
print(f'📊 选择的数据集: {info["name"]}')
print(f'   标签数量: {info["labels"]}')
print(f'   准确率(mIoU): {info["miou"]}')
print(f'   说明: {info["desc"]}')


📊 选择的数据集: LIP (Look Into Person)
   标签数量: 20
   准确率(mIoU): 59.36%
   说明: 最大数据集，包含帽子、头发、手套、上衣、裤子等20个部位


In [4]:
# ========== 步骤 4: 下载预训练模型 ==========
print(f'📥 下载 {dataset.upper()} 预训练模型...')
print('⏳ 模型大小约 260MB，需要 1-2 分钟...\n')

!pip install -q gdown

import gdown

# 模型下载链接
model_urls = {
    'lip': 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH',
    'atr': 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP',
    'pascal': 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'
}

url = model_urls[dataset]
output_path = 'checkpoints/final.pth'

gdown.download(url, output_path, quiet=False)

print(f'\n✅ 模型下载完成！保存在: {output_path}')


📥 下载 LIP 预训练模型...
⏳ 模型大小约 260MB，需要 1-2 分钟...



Downloading...
From (original): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=4e0e5655-1336-44d7-8df5-a2b280fb5ad3
To: /content/checkpoints/final.pth
100%|██████████| 267M/267M [00:01<00:00, 148MB/s]


✅ 模型下载完成！保存在: checkpoints/final.pth


In [5]:
# ========== 步骤 5: 上传图片 ==========
print('📤 请选择要处理的图片（可以多选）...\n')

from google.colab import files
import shutil
import os

# 上传文件
uploaded = files.upload()

# 移动到 inputs 文件夹
for filename in uploaded.keys():
    src = filename
    dst = f'inputs/{filename}'
    shutil.move(src, dst)
    print(f'✅ 已上传: {filename}')

# 显示上传的图片
print(f'\n📊 共上传 {len(uploaded)} 张图片')
print('图片列表:', list(uploaded.keys()))


📤 请选择要处理的图片（可以多选）...



Saving c22b87643cfc465e8e4fcf233ea40bf1.png to c22b87643cfc465e8e4fcf233ea40bf1.png
✅ 已上传: c22b87643cfc465e8e4fcf233ea40bf1.png

📊 共上传 1 张图片
图片列表: ['c22b87643cfc465e8e4fcf233ea40bf1.png']


In [6]:
# ========== 步骤 6: 运行人体解析 ==========
print('🚀 开始处理图片...')
print(f'📊 使用数据集: {dataset.upper()}')
print('⏳ 每张图片约需 1-3 秒...\n')

import time
import os

# 检查输入文件
input_count = len([f for f in os.listdir('inputs') if f.endswith(('.png', '.jpg', '.jpeg'))])
print(f'📂 inputs 文件夹中有 {input_count} 张图片')

start_time = time.time()

# 运行推理
!python simple_extractor.py --dataset {dataset} --model-restore checkpoints/final.pth --input-dir inputs --output-dir outputs

elapsed = time.time() - start_time

# 检查输出结果
output_count = len([f for f in os.listdir('outputs')])
print(f'\n📂 outputs 文件夹中生成了 {output_count} 个文件')

if output_count > 0:
    print('📋 输出文件列表:')
    for f in os.listdir('outputs'):
        print(f'   - {f}')
    print(f'\n✅ 处理完成！')
else:
    print('⚠️  警告: outputs 文件夹为空，请检查上面是否有错误信息')

print(f'⏱️  总耗时: {elapsed:.1f} 秒')


🚀 开始处理图片...
📊 使用数据集: LIP
⏳ 每张图片约需 1-3 秒...

📂 inputs 文件夹中有 1 张图片
python3: can't open file '/content/simple_extractor.py': [Errno 2] No such file or directory

📂 outputs 文件夹中生成了 0 个文件
⚠️  警告: outputs 文件夹为空，请检查上面是否有错误信息
⏱️  总耗时: 0.1 秒


In [7]:
# ========== 步骤 7: 显示结果 ==========
print('🖼️  显示处理结果...\n')

from PIL import Image
import matplotlib.pyplot as plt
import os

# 获取所有文件
input_files = sorted([f for f in os.listdir('inputs') if f.endswith(('.png', '.jpg', '.jpeg'))])
all_output_files = sorted(os.listdir('outputs'))
output_files = sorted([f for f in all_output_files if f.endswith('.png')])

print(f'📊 找到 {len(input_files)} 张原图')
print(f'📊 outputs 文件夹中有 {len(all_output_files)} 个文件')
print(f'📊 其中 {len(output_files)} 个是 PNG 图片\n')

# 如果没有输出文件，显示警告
if len(output_files) == 0:
    print('❌ 错误: outputs 文件夹中没有找到结果图片！')
    print('可能的原因:')
    print('  1. 步骤6的推理过程出错了')
    print('  2. 请检查步骤6是否有红色错误信息')
    print('  3. 确认已经正确安装了 PyTorch 1.7.1')
    print('\n💡 建议: 重新运行步骤6，仔细查看输出信息')
else:
    print('📋 输出文件:')
    for f in output_files[:5]:  # 只显示前5个
        print(f'   {f}')
    if len(output_files) > 5:
        print(f'   ... 还有 {len(output_files)-5} 个文件')
    print()

    # 显示每组对比
    success_count = 0
    for idx, inp_file in enumerate(input_files, 1):
        # 尝试多种文件名匹配方式
        base_name = os.path.splitext(inp_file)[0]
        possible_names = [
            inp_file.replace('.jpg', '.png').replace('.jpeg', '.png'),  # 直接替换扩展名
            base_name + '.png',  # 基础名 + .png
            base_name + '_vis.png',  # 有些版本会添加 _vis 后缀
        ]

        out_file = None
        for possible in possible_names:
            if possible in output_files:
                out_file = possible
                break

        if out_file is None:
            print(f'⚠️  未找到 {inp_file} 的输出结果')
            print(f'    尝试过的文件名: {possible_names}')
            continue

        # 创建子图
        fig, axes = plt.subplots(1, 2, figsize=(14, 7))
        fig.suptitle(f'图片 {idx}/{len(input_files)}: {inp_file}', fontsize=14, fontweight='bold')

        # 显示原图
        img_input = Image.open(f'inputs/{inp_file}')
        axes[0].imshow(img_input)
        axes[0].set_title('原图', fontsize=12)
        axes[0].axis('off')

        # 显示解析结果
        img_output = Image.open(f'outputs/{out_file}')
        axes[1].imshow(img_output)
        axes[1].set_title(f'人体解析结果 ({dataset.upper()})', fontsize=12)
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()
        print(f'  ✅ {inp_file} → {out_file}')
        print(f'  尺寸: {img_input.size[0]}x{img_input.size[1]} 像素')
        print()
        success_count += 1

    print(f'✅ 成功显示 {success_count}/{len(input_files)} 组结果！')


🖼️  显示处理结果...

📊 找到 1 张原图
📊 outputs 文件夹中有 0 个文件
📊 其中 0 个是 PNG 图片

❌ 错误: outputs 文件夹中没有找到结果图片！
可能的原因:
  1. 步骤6的推理过程出错了
  2. 请检查步骤6是否有红色错误信息
  3. 确认已经正确安装了 PyTorch 1.7.1

💡 建议: 重新运行步骤6，仔细查看输出信息


In [8]:
# ========== 步骤 8: 下载结果（可选） ==========
print('📦 打包并下载处理结果...\n')

import shutil
from google.colab import files
import os

# 打包 outputs 文件夹
archive_name = f'human_parsing_results_{dataset}'
shutil.make_archive(archive_name, 'zip', 'outputs')

# 显示文件信息
zip_size = os.path.getsize(f'{archive_name}.zip') / (1024 * 1024)  # MB
file_count = len([f for f in os.listdir('outputs') if f.endswith('.png')])

print(f'📊 压缩包信息:')
print(f'   文件名: {archive_name}.zip')
print(f'   大小: {zip_size:.2f} MB')
print(f'   包含文件: {file_count} 张图片')
print(f'\n🔽 开始下载...')

# 下载
files.download(f'{archive_name}.zip')

print('✅ 下载完成！')


📦 打包并下载处理结果...

📊 压缩包信息:
   文件名: human_parsing_results_lip.zip
   大小: 0.00 MB
   包含文件: 0 张图片

🔽 开始下载...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 下载完成！


---

## 📚 数据集详细说明

### 🎯 LIP (Look Into Person)
- **标签数**: 20个
- **准确率**: mIoU 59.36%
- **数据量**: 50,000+ 图片
- **标签列表**:
  - Background（背景）
  - Hat（帽子）, Hair（头发）, Glove（手套）, Sunglasses（太阳镜）
  - Upper-clothes（上衣）, Dress（连衣裙）, Coat（外套）, Pants（裤子）
  - Socks（袜子）, Jumpsuits（连体衣）, Scarf（围巾）, Skirt（裙子）
  - Face（脸）
  - Left-arm（左臂）, Right-arm（右臂）
  - Left-leg（左腿）, Right-leg（右腿）
  - Left-shoe（左鞋）, Right-shoe（右鞋）

### 👔 ATR (A person re-identification dataset)
- **标签数**: 18个
- **准确率**: mIoU 82.29%
- **数据量**: 17,000+ 图片
- **适用场景**: 时尚AI、服装识别
- **标签列表**:
  - Background（背景）
  - Hat（帽子）, Hair（头发）, Sunglasses（太阳镜）
  - Upper-clothes（上衣）, Skirt（裙子）, Pants（裤子）, Dress（连衣裙）
  - Belt（腰带）, Bag（包）, Scarf（围巾）
  - Left-shoe（左鞋）, Right-shoe（右鞋）
  - Face（脸）
  - Left-leg（左腿）, Right-leg（右腿）
  - Left-arm（左臂）, Right-arm（右臂）

### 🏃 Pascal-Person-Part
- **标签数**: 7个
- **准确率**: mIoU 71.46%
- **数据量**: 3,000+ 图片
- **适用场景**: 身体部位分割（简化版）
- **标签列表**:
  - Background（背景）
  - Head（头部）
  - Torso（躯干）
  - Upper Arms（上臂）
  - Lower Arms（下臂）
  - Upper Legs（大腿）
  - Lower Legs（小腿）

---

## 💡 常见问题

### Q: 为什么要用 PyTorch 1.7.1？
**A**: 代码使用了旧版 API，新版 PyTorch (2.0+) 会报编译错误。1.7.1 完全兼容。

### Q: 可以处理视频吗？
**A**: 可以！先把视频拆成帧，批量处理后再合成视频。

### Q: 报错 "CUDA out of memory"？
**A**: 图片太大。解决方法：
- 降低图片分辨率
- 减少批处理数量
- 或升级到更大的 GPU

### Q: 结果不准确怎么办？
**A**: 尝试：
- 换其他数据集（如 ATR 对时尚服装更准）
- 确保人物占图片主要部分
- 避免过度遮挡

---

## 🎉 完成！

如有问题，请访问：
- 项目地址: https://github.com/likunpeng0127/salf
- 原始论文: [Self-Correction for Human Parsing](https://arxiv.org/abs/1910.09777)
